In [ ]:
import yaml

# Load the YAML file
with open('assets/yml/vocabulary.yml', 'r') as file:
    vocab = yaml.safe_load(file)

# Convert to Quarto markdown
qmd_content = "# Vocabulary\n\n"
for key, value in vocab.items():
    qmd_content += f"## {key}\n\n"
    qmd_content += f"{value}\n\n"

# Save the markdown content to a file
with open('vocabulary.qmd', 'w') as file:
    file.write(qmd_content)

print("Conversion complete. The markdown file is saved as 'vocabulary.qmd'.")

Conversion complete. The markdown file is saved as 'vocabulary.qmd'.
